# Simulation of a quantum algorithm for the inverse travel time problem for graphs

This notebook produced the plots in Figure 3 in Appendix A of *Quantum computing algorithms for inverse problems on graphs and an NP-complete inverse problem* (https://arxiv.org/abs/2306.05253) by Joonas Ilmavirta, Matti Lassas, Jinpeng Lu, Lauri Oksanen, and Lauri Ylinen. 

In [ ]:
from indexing import *
from circuits import *
from expected import *

from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator
from qiskit.visualization import plot_distribution

import time
import math

def oracle(imat, pairs_known=None):
    ix, ds = dmat2ds(dmat_expected(imat))
    if pairs_known is None:
        ds_known = ds
    else:
        ix_known = set([ix.edge(*p) for p in pairs_known])
        ds_known = [(ix, d) for (ix, d) in ds if ix in ix_known]
    state_qubits = range(ix.num_edges)
    return Dist_check(ix, ds_known), list(state_qubits)

def build_circuit(imat, pairs_known, n_iterations):
    grover_oracle, state_qubits = oracle(imat, pairs_known)
    grover_op = GroverOperator(grover_oracle, reflection_qubits=state_qubits)
    
    qc = QuantumCircuit(grover_op.num_qubits, len(state_qubits))
    # Put target qubit into state |->
    qc.x(-1)
    qc.h(-1)
    # Create even superposition of all basis states
    qc.h(state_qubits)
    # Apply Grover operator n_iterations times
    qc.compose(grover_op.power(n_iterations), inplace=True)
    # Measure state qubits
    qc.measure(state_qubits, state_qubits)
    return qc

def run_circuit(qc):
    from qiskit.primitives import Sampler
    sampler = Sampler()
    job = sampler.run(qc)
    result = job.result()
    return result.quasi_dists[0].binary_probabilities()

def simulate_circuit(qc):
    start = time.time()
    quasi_dist = run_circuit(qc)
    end = time.time()
    print('elapsed time in minutes:', (end - start) / 60)
    return quasi_dist

def save_plot(plt, filename):
    # For simulation, Quasi-Probability is the same as Probability, so change label of y-axis:
    ax = plt.gca()
    ax.set_ylabel('Probability', fontsize = 14)
    ax.set_xlabel('Measurement', fontsize = 14)
    ax.tick_params(axis='both', labelrotation=0, labelsize=11)
    display(plt)
    plt.savefig(filename, bbox_inches = "tight", transparent=True)

def run_instance(imat, pairs_known, n_iterations, n_to_keep, filename, figsize):
    qc = build_circuit(imat, pairs_known, n_iterations)
    display(qc.draw('mpl', style="iqp"))
    print('number of qubits in the circuit: ', qc.num_qubits)

    quasi_dist = simulate_circuit(qc)
    if n_to_keep == 'all':
        plt = plot_distribution(quasi_dist, figsize=figsize)
    else:
        plt = plot_distribution(quasi_dist, number_to_keep=n_to_keep, figsize=figsize)
    save_plot(plt, filename)

## Instance A

In [ ]:
# Test with
# 1 - 2 - 0
imat = np.full((3,3), False)
imat[0, 2] = True
imat[1, 2] = True
# Pairs of nodes for which distances are known 
pairs_known = [(0,1)]

n_iterations = 2
n_to_keep = 'all'
filename = 'instance_A.png'
figsize = (8,5)
run_instance(imat, pairs_known, n_iterations, n_to_keep, filename, figsize)

## Instance B

In [ ]:
# Test with
# 1   0
# |\ /
# | 3
# |/
# 2
imat = np.full((4,4), False)
imat[0, 3] = True
imat[1, 2] = True
imat[1, 3] = True
imat[2, 3] = True
# Pairs of nodes for which distances are known 
pairs_known = [(0,1), (0,2), (1, 2)]

n_iterations = 6
n_to_keep = 1
filename = 'instance_B.png'
figsize = (2.5,5)
run_instance(imat, pairs_known, n_iterations, n_to_keep, filename, figsize)

## Instance C

In [ ]:
# Test with
# 1   0
# |\ /|
# | 4 |
# |   |
# 2---3
imat = np.full((5,5), False)
imat[0, 3] = True
imat[0, 4] = True
imat[1, 2] = True
imat[1, 4] = True
imat[2, 3] = True
# Pairs of nodes for which distances are known 
pairs_known = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]

n_iterations = 13
n_to_keep = 4
filename = 'instance_C.png'
figsize = (8,5)
run_instance(imat, pairs_known, n_iterations, n_to_keep, filename, figsize)

## Instance D

In [ ]:
# Test with
# 1    0
#  \  /
#   4
#  / \
# 2   3
imat = np.full((5,5), False)
imat[0, 4] = True
imat[1, 4] = True
imat[2, 4] = True
imat[3, 4] = True
# Pairs of nodes for which distances are known 
pairs_known = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]

n_iterations = 25
n_to_keep = 1
filename = 'instance_D.png'
figsize = (2.5,5)
run_instance(imat, pairs_known, n_iterations, n_to_keep, filename, figsize)

## Record of system information and version numbers of packages

In [ ]:
from Systeminfo.system_Info import SystemInfo

system_info = SystemInfo()
all_info = system_info.get_all_info()

# Print the system information
print("===== System Information =====")
for key, value in all_info["System Information"].items():
    print(f"{key}: {value}")

# Print the CPU information
print("\n===== CPU Information =====")
for key, value in all_info["CPU Information"].items():
    print(f"{key}: {value}")

# Print the GPU information if available
if "GPU Information" in all_info:
    print("\n===== GPU Information =====")
    for gpu_info in all_info["GPU Information"]:
        for key, value in gpu_info.items():
            print(f"{key}: {value}")


In [ ]:
pip list